<a href="https://colab.research.google.com/github/samuel0711/Desafio/blob/main/desafio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import requests
import json
import pandas as pd
import psycopg2
import datetime as dt

import matplotlib.pyplot as plt 

pd.set_option('display.max_columns', 50)

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
# Função para criar conexão no banco
def conecta_db():
  con = psycopg2.connect(host='ec2-3-222-74-92.compute-1.amazonaws.com', 
                         database='dfo2vqe9hs2knm',
                         user='zgbodaynfjorzt',
                         port='5432', 
                         password='d2b11c0c6039df67f39d22f680fb92a843a1b652e3d5b692f6670c0a2abc2804')
  return con

In [4]:
#Consulta em tabela específica
def pegar_tabela(tabela):
  con = conecta_db()
  # cria um cursor
  cur = con.cursor()
  #construindo query
  query = 'SELECT * FROM '+tabela
  cur.execute(query)
  # pegar todos os dados da query e armazenar dentro de uma lista
  data = cur.fetchall()

  registros = []
  for rec in data:
    registros.append(rec)
  con.close()

  #print(registros[0])
  return registros

#Consulta sobre nomes das colunas de uma tabela
def pegar_colunas(tabela):
  con = conecta_db()
  # cria um cursor
  cur = con.cursor()
  #construindo query
  query = "SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = '"+tabela+"'"
  cur.execute(query)
  col_names = cur.fetchall()
  con.close()

  col_names = [col[0] for col in col_names]
  #print(col_names)
  return col_names

# Função para consulta de versão e tabelas do banco
def consultar_db():
  con = conecta_db()
  # cria um cursor
  cur = con.cursor()
  # executa uma declaração
  print('Versão do banco de dados PostgreSQL:')
  cur.execute ('SELECT version()')
  # exibir a versão do servidor de banco de dados PostgreSQL
  db_version = cur.fetchone()
  print(db_version)

  #verificando tabelas disponíveis
  cur.execute ("SELECT * FROM pg_catalog.pg_tables WHERE schemaname != 'pg_catalog' AND schemaname != 'information_schema'")
  tables = cur.fetchall()
  for rec in tables:
    print(rec)

#Retorna as consultas e monta um dataframe
def fazer_consulta(tabela):
  registros = pegar_tabela(tabela)
  colunas = pegar_colunas(tabela)
  df = pd.DataFrame(registros, columns=colunas)
  return df

In [5]:
consultar_db()

Versão do banco de dados PostgreSQL:
('PostgreSQL 14.4 (Ubuntu 14.4-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0, 64-bit',)
('public', 'forms', 'zgbodaynfjorzt', None, True, False, True, False)
('public', 'order_items', 'zgbodaynfjorzt', None, True, False, True, False)
('public', 'orders', 'zgbodaynfjorzt', None, True, False, True, False)


In [6]:
forms = fazer_consulta('forms')
order_items = fazer_consulta('order_items')
orders = fazer_consulta('orders')

In [7]:
x = forms[['id_form','id_cliente','data']]
x['data'] = x['data'].dt.date
forms.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id_form,id_cliente,data,sistema_operacional,navegador,localizacao,locale,cidade,estado,qtde_lavagem_semana,utm_source,tipo_cabelo,comprimento,tipo_fios,procedimentos,tempo_procedimento,dieta,atividade_fisica,frequencia_estresse,caracteristica,faixa_etaria,efeitos_desejados,fragancia
0,c7d453c9c01223a42968d9fe136047,d2620a3e6f00a6dc6e88,2022-01-03 15:42:22,iOS,Safari,"São Paulo, Sao Paulo",BR,SÃO PAULO,SAO PAULO,4,None,Tipo 2,Raíz exposta,Pontas duplas,Nenhum,Há 2 semanas,Celíaca,Frequente,Nunca,Característica 1,25 a 34,Efeito 7,Rosas
1,72830da153824af486be15cd48b3f3,9a48802d8290d19471f3,2022-01-04 08:39:53,Windows,Chrome,"Ribeirão Preto, Sao Paulo",BR,RIBEIRÃO PRETO,SAO PAULO,7,None,Tipo 4,Muito curto,Perfeitos,Todos acima,Há mais de 1 mês,Nenhuma restrição,Nunca,Nunca,Característica 6,55 a 64,Efeito 1,Eucalipto
2,d101fc0075dac29db1168cfc61ad44,c3344fad450ee424700d,2022-01-04 19:34:05,Android,Chrome,"São Paulo, Sao Paulo",BR,SÃO PAULO,SAO PAULO,2,None,Tipo 1,Extra longo,Rebeldes,Todos acima,Há mais de 1 mês,Nenhuma restrição,Raramente,Regularmente,Característica 7,2 a 10,Efeito 7,Rosas
3,0957fb9d7be36b4d079b6582bb6785,8105e34124e90300d8b5,2022-01-05 18:29:41,Android,Chrome,"Manchester, England",BR,MANCHESTER,ENGLAND,5,None,Tipo 1,Extra longo,Rebeldes,Todos acima,Há mais de 1 mês,Celíaca,Raramente,Regularmente,Característica 7,25 a 34,Efeito 7,Rosas
4,ba73bdcc6c063ab08b533eae5505eb,636382f3f545eec3fe97,2022-01-07 11:44:31,Android,Chrome,"Tijucas, Santa Catarina",BR,TIJUCAS,SANTA CATARINA,4,facebook,Tipo 3,Muito curto,Rebeldes,Tintura,Há 1 semana,Celíaca,Sempre,Nunca,Característica 10,18 a 24,Efeito 7,Eucalipto


In [8]:
order_items.head()

,id,id_pedido,id_form,id_produto,qtde,valor
0,423803,dd7dd09eb757eab,62296554a44f818ced0d623434e26b,509d,2,74.95
1,423804,dd7dd09eb757eab,62296554a44f818ced0d623434e26b,e820,1,74.95
2,423805,dd7dd09eb757eab,62296554a44f818ced0d623434e26b,d346,2,59.90
3,423806,dd7dd09eb757eab,62296554a44f818ced0d623434e26b,d2e0,1,64.90
4,423807,46a72a1e08c387e,25e1f26f2ec67b0cbefde74337185a,509d,2,74.95


In [9]:
y = orders[['id_pedido','id_cliente','data_pedido']]
y['data_pedido'] = y['data_pedido'].dt.date

orders.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id_pedido,id_cliente,data_pedido,status_pagamento,tipo_pagamento,valor_total,valor_desconto,frete_gratis,cidade,estado,locale,utm_source
0,c677716e0f2901f,001b7aed7554265e4e5f,2022-03-22 21:10:49,Entregue,CARTÃO,198.63,0.00,True,Osasco,SP,BR,google
1,df063dd3f6b1641,d107878c463e7c969ce6,2022-03-22 21:47:08,Entregue,CARTÃO,198.63,0.00,True,São Paulo,SP,BR,influencer
2,dd69f88884e45a8,44845cf79366dbcecf00,2022-03-08 22:50:22,Entregue,CARTÃO,260.96,0.00,True,Bauru,SP,BR,facebook
3,2c556256ec3a77e,2aa1a61cfa7be6c51167,2022-01-21 14:53:44,Entregue,CARTÃO,225.24,0.00,False,Ananindeua,PA,BR,facebook
4,0262be22fc1c6f0,10c138383539f471831f,2022-03-08 22:50:24,Entregue,CARTÃO,199.31,0.00,True,Saquarema,RJ,BR,facebook


In [10]:
print(forms.shape, order_items.shape, orders.shape) #order items = orders, porém com granularidade de produtos por registro

(427950, 23) (211663, 6) (55078, 12)


In [17]:
orders[orders.id_cliente == 'd2620a3e6f00a6dc6e88']

,id_pedido,id_cliente,data_pedido,status_pagamento,tipo_pagamento,valor_total,valor_desconto,frete_gratis,cidade,estado,locale,utm_source
35038,5d2301cb6bf9d7c,d2620a3e6f00a6dc6e88,2022-01-03 15:47:35,Entregue,CARTÃO,124.03,0.00,False,São Paulo,SP,BR,None


In [24]:
df = x.merge(y, on=['id_cliente'], how='left')
df['minus'] = df['data_pedido'] - df['data']

#df

In [15]:
df[df['id_cliente'] == 'b6c0b82cc271fd1203fc']

,id_form,id_cliente,data,id_pedido,data_pedido,minus
35,b30e92d3b00d98828e057ca0ef10cb,b6c0b82cc271fd1203fc,2022-01-25,60bd3a78d0516f2,2022-04-06,71 days
36,b30e92d3b00d98828e057ca0ef10cb,b6c0b82cc271fd1203fc,2022-01-25,54ee2d459feb87f,2022-01-28,3 days
37,b30e92d3b00d98828e057ca0ef10cb,b6c0b82cc271fd1203fc,2022-01-25,970260f5e1a64c6,2022-04-11,76 days
38,b30e92d3b00d98828e057ca0ef10cb,b6c0b82cc271fd1203fc,2022-01-25,31130e565bef07d,2022-04-11,76 days
39,b30e92d3b00d98828e057ca0ef10cb,b6c0b82cc271fd1203fc,2022-01-25,aaf5feb75e0da96,2022-04-12,77 days
...,...,...,...,...,...,...
484374,c172d08a3a36b903cd49f6af8da9c3,b6c0b82cc271fd1203fc,2022-01-27,3c5afd1165df4f1,2022-06-14,138 days
484375,c172d08a3a36b903cd49f6af8da9c3,b6c0b82cc271fd1203fc,2022-01-27,3d8c4f8779d80d9,2022-06-14,138 days
484376,c172d08a3a36b903cd49f6af8da9c3,b6c0b82cc271fd1203fc,2022-01-27,e02be23c1715d8a,2022-06-21,145 days
484377,c172d08a3a36b903cd49f6af8da9c3,b6c0b82cc271fd1203fc,2022-01-27,a7e3b36b221d402,2022-04-12,75 days


In [23]:
forms.loc[(forms.id_cliente == 'b6c0b82cc271fd1203fc') & (forms.data.dt.month == 5)].sort_values(by='data')

,id_form,id_cliente,data,sistema_operacional,navegador,localizacao,locale,cidade,estado,qtde_lavagem_semana,utm_source,tipo_cabelo,comprimento,tipo_fios,procedimentos,tempo_procedimento,dieta,atividade_fisica,frequencia_estresse,caracteristica,faixa_etaria,efeitos_desejados,fragancia
288706,f5dcf9260254876a2e6105f644307a,b6c0b82cc271fd1203fc,2022-05-02 14:56:09,Windows,Chrome,"Itatiba, Sao Paulo",BR,ITATIBA,SAO PAULO,7,None,Tipo 4,Muito curto,Perfeitos,Nenhum,Não houve procedimentos,Nenhuma restrição,N/C,Nunca,Característica 9,65+,Efeito 1,Eucalipto
256702,6a90cdb0fddc194bc35a2708ab52c4,b6c0b82cc271fd1203fc,2022-05-03 09:19:49,Windows,Chrome,"Itatiba, Sao Paulo",BR,ITATIBA,SAO PAULO,7,None,Tipo 4,Muito curto,Perfeitos,Nenhum,Não houve procedimentos,Intolerante a lactose,Nunca,Nunca,Característica 9,35 a 44,Efeito 1,Eucalipto
291187,ee33f5a07d9b06ddd02f4a89ed11d9,b6c0b82cc271fd1203fc,2022-05-03 09:24:40,Windows,Chrome,"Itatiba, Sao Paulo",BR,ITATIBA,SAO PAULO,3,None,Tipo 4,Muito curto,Perfeitos,Nenhum,Não houve procedimentos,Nenhuma restrição,Nunca,Nunca,"Característica 6, Característica 9",11 a 17,Efeito 4,Eucalipto
256832,7e75ab2ce4a71b7df6689bd79d7641,b6c0b82cc271fd1203fc,2022-05-03 09:37:05,Windows,Chrome,"Itatiba, Sao Paulo",BR,ITATIBA,SAO PAULO,7,None,Tipo 1,Muito curto,Perfeitos,Todos acima,Há mais de 1 mês,Nenhuma restrição,Nunca,Nunca,Característica 6,55 a 64,Efeito 1,Eucalipto
256955,be482f5585c80cd6e7464d756daf23,b6c0b82cc271fd1203fc,2022-05-03 09:38:39,Windows,Chrome,"Itatiba, Sao Paulo",BR,ITATIBA,SAO PAULO,7,None,Tipo 4,Muito curto,Perfeitos,Todos acima,Há mais de 1 mês,Nenhuma restrição,Nunca,Nunca,Característica 8,55 a 64,Efeito 4,Eucalipto
256970,ea8fab87e53dc0cf42f71757f40c2b,b6c0b82cc271fd1203fc,2022-05-03 09:39:58,Windows,Chrome,"Itatiba, Sao Paulo",BR,ITATIBA,SAO PAULO,7,None,Tipo 4,Muito curto,Perfeitos,Todos acima,Há mais de 1 mês,Nenhuma restrição,Nunca,Nunca,Característica 6,55 a 64,Efeito 1,Flores do campo
257609,dc7658eaf0ad36a690f9083f61b797,b6c0b82cc271fd1203fc,2022-05-03 09:40:28,Windows,Chrome,"Itatiba, Sao Paulo",BR,ITATIBA,SAO PAULO,7,None,Tipo 4,Muito curto,Perfeitos,Nenhum,Não houve procedimentos,Nenhuma restrição,Nunca,Nunca,Característica 9,55 a 64,Efeito 3,Eucalipto
257732,259c3bdafad4d394f72231e9e1d9d9,b6c0b82cc271fd1203fc,2022-05-03 09:51:04,Windows,Chrome,"Itatiba, Sao Paulo",BR,ITATIBA,SAO PAULO,7,None,Tipo 4,Muito curto,Perfeitos,Todos acima,Há mais de 1 mês,Nenhuma restrição,Nunca,Nunca,Característica 6,45 a 54,Efeito 1,Eucalipto
257796,e75ea60f7ef25e89af4de56640cba6,b6c0b82cc271fd1203fc,2022-05-03 09:57:21,Windows,Chrome,"Itatiba, Sao Paulo",BR,ITATIBA,SAO PAULO,7,None,Tipo 4,Muito curto,Perfeitos,Exposição ao sol,Não houve procedimentos,Nenhuma restrição,Nunca,Nunca,Característica 9,55 a 64,Efeito 1,Eucalipto
257841,d3b01a7638289a155a4b7b179cfeaf,b6c0b82cc271fd1203fc,2022-05-03 09:58:31,Windows,Chrome,"Itatiba, Sao Paulo",BR,ITATIBA,SAO PAULO,3,None,Tipo 1,Muito curto,Perfeitos,Todos acima,Há mais de 1 mês,Intolerante a lactose,Nunca,Nunca,Característica 6,11 a 17,Efeito 1,Flores do campo


In [20]:
orders.loc[(orders.id_cliente == 'b6c0b82cc271fd1203fc') & (orders.status_pagamento != 'Cancelado')].sort_values(by='data_pedido')

,id_pedido,id_cliente,data_pedido,status_pagamento,tipo_pagamento,valor_total,valor_desconto,frete_gratis,cidade,estado,locale,utm_source
35809,3de41ce9e954cf4,b6c0b82cc271fd1203fc,2022-05-16 09:01:58,Pagamento aprovado,CARTÃO,274.25,0.00,False,São Paulo,SP,BR,None


In [26]:
order_items[order_items.id_pedido == '3de41ce9e954cf4']

,id,id_pedido,id_form,id_produto,qtde,valor
191159,614962,3de41ce9e954cf4,cca2dd772b2989216d22bf0d422c25,d2e0,1,64.90
191160,614963,3de41ce9e954cf4,cca2dd772b2989216d22bf0d422c25,509d,1,74.95
191161,614964,3de41ce9e954cf4,cca2dd772b2989216d22bf0d422c25,e820,1,74.95
191162,614965,3de41ce9e954cf4,cca2dd772b2989216d22bf0d422c25,d346,1,59.90


In [27]:
forms[forms.id_form=='cca2dd772b2989216d22bf0d422c25']

,id_form,id_cliente,data,sistema_operacional,navegador,localizacao,locale,cidade,estado,qtde_lavagem_semana,utm_source,tipo_cabelo,comprimento,tipo_fios,procedimentos,tempo_procedimento,dieta,atividade_fisica,frequencia_estresse,caracteristica,faixa_etaria,efeitos_desejados,fragancia
76881,cca2dd772b2989216d22bf0d422c25,b6c0b82cc271fd1203fc,2022-01-26 08:36:49,iOS,Safari,"Ribeirão Preto, Sao Paulo",BR,RIBEIRÃO PRETO,SAO PAULO,4,None,Tipo 3,Longo,Perfeitos,Exposição ao sol,Não houve procedimentos,Leve,Raramente,Nunca,Característica 6,25 a 34,Efeito 2,Rosas
